Test 1

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator)

Epoch 1/5
313/313 [==============================] - 6112s 19s/step - loss: 0.2064 - accuracy: 0.9158 - val_loss: 0.1515 - val_accuracy: 0.9445
Epoch 2/5
313/313 [==============================] - 128s 410ms/step - loss: 0.1143 - accuracy: 0.9553 - val_loss: 0.2767 - val_accuracy: 0.9215
Epoch 3/5
313/313 [==============================] - 128s 408ms/step - loss: 0.0975 - accuracy: 0.9621 - val_loss: 0.1716 - val_accuracy: 0.9330
Epoch 4/5
313/313 [==============================] - 127s 407ms/step - loss: 0.0779 - accuracy: 0.9676 - val_loss: 0.2692 - val_accuracy: 0.9175
Epoch 5/5
313/313 [==============================] - 127s 407ms/step - loss: 0.0711 - accuracy: 0.9702 - val_loss: 0.4681 - val_accuracy: 0.8705


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 1031s 17s/step - loss: 1.2067 - accuracy: 0.7305
Test Loss: 1.2067334651947021
Test Accuracy: 0.7304999828338623


Test 2

* Data Augmentation: I added more aggressive data augmentation techniques such as rotation, width and height shifts, shear, zoom, horizontal flip, and brightness adjustment.
* Fine-tuning: I used a dropout layer to prevent overfitting and added a ReduceLROnPlateau callback for learning rate scheduling during training.
* Learning Rate Scheduling: The learning rate will decrease by a factor of 0.2 if the validation loss does not improve for 2 epochs. This helps the model to converge more effectively.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]  # Add brightness adjustment
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 190s 506ms/step - loss: 0.2123 - accuracy: 0.9132 - val_loss: 0.5256 - val_accuracy: 0.8760 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 157s 499ms/step - loss: 0.1221 - accuracy: 0.9535 - val_loss: 0.3531 - val_accuracy: 0.9065 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 158s 506ms/step - loss: 0.1061 - accuracy: 0.9588 - val_loss: 0.1973 - val_accuracy: 0.9245 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 158s 503ms/step - loss: 0.0926 - accuracy: 0.9619 - val_loss: 0.2017 - val_accuracy: 0.9350 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 159s 508ms/step - loss: 0.0752 - accuracy: 0.9698 - val_loss: 0.1506 - val_accuracy: 0.9420 - lr: 0.0010


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 6s 97ms/step - loss: 0.4591 - accuracy: 0.8755
Test Loss: 0.45907220244407654
Test Accuracy: 0.8755000233650208


Test 3

* We fine-tune the top layers of the pre-trained model by unfreezing them and continuing training with a lower learning rate.
* Learning rate scheduling is continued to adapt to the fine-tuning process.
* Dropout regularization is used to reduce overfitting during training.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]  # Add brightness adjustment
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 162s 497ms/step - loss: 0.6051 - accuracy: 0.6713 - val_loss: 0.5309 - val_accuracy: 0.7335 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 153s 487ms/step - loss: 0.5342 - accuracy: 0.7249 - val_loss: 0.5103 - val_accuracy: 0.7430 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 154s 490ms/step - loss: 0.5248 - accuracy: 0.7363 - val_loss: 0.4899 - val_accuracy: 0.7580 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 154s 493ms/step - loss: 0.5235 - accuracy: 0.7382 - val_loss: 0.4935 - val_accuracy: 0.7570 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 154s 491ms/step - loss: 0.5121 - accuracy: 0.7400 - val_loss: 0.4837 - val_accuracy: 0.7660 - lr: 0.0010


In [ ]:
# Fine-tune the top layers of the model
for layer in model.layers[-20:]:
    layer.trainable = True

In [ ]:
# Adjust learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),  # Lower learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with fine-tuning
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 165s 500ms/step - loss: 0.3405 - accuracy: 0.8460 - val_loss: 0.3980 - val_accuracy: 0.8465 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 153s 490ms/step - loss: 0.2293 - accuracy: 0.9054 - val_loss: 0.2820 - val_accuracy: 0.8805 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 154s 491ms/step - loss: 0.1962 - accuracy: 0.9182 - val_loss: 0.5044 - val_accuracy: 0.8200 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 155s 496ms/step - loss: 0.1785 - accuracy: 0.9301 - val_loss: 0.2666 - val_accuracy: 0.9020 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 153s 490ms/step - loss: 0.1561 - accuracy: 0.9368 - val_loss: 0.3574 - val_accuracy: 0.8670 - lr: 0.0010


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 7s 104ms/step - loss: 0.6643 - accuracy: 0.7895
Test Loss: 0.6643425822257996
Test Accuracy: 0.7894999980926514


Test 4
* We fine-tune more layers of the pre-trained model by unfreezing additional layers.
* We adjust the learning rate even further for fine-tuning to ensure better convergence.
* We also add more data augmentation techniques, such as fill_mode, to provide the model with more diverse training samples.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Add brightness adjustment
    fill_mode='nearest'  # Add fill mode
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 1668s 5s/step - loss: 0.6051 - accuracy: 0.6713 - val_loss: 0.5309 - val_accuracy: 0.7335 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 151s 482ms/step - loss: 0.5342 - accuracy: 0.7249 - val_loss: 0.5103 - val_accuracy: 0.7430 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 149s 476ms/step - loss: 0.5248 - accuracy: 0.7363 - val_loss: 0.4899 - val_accuracy: 0.7580 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 150s 479ms/step - loss: 0.5235 - accuracy: 0.7382 - val_loss: 0.4935 - val_accuracy: 0.7570 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 151s 484ms/step - loss: 0.5121 - accuracy: 0.7400 - val_loss: 0.4837 - val_accuracy: 0.7660 - lr: 0.0010


In [ ]:
# Fine-tune more layers of the model
for layer in model.layers[-100:]:
    layer.trainable = True

In [ ]:
# Adjust learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001),  # Lower learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with fine-tuning
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 167s 480ms/step - loss: 0.2354 - accuracy: 0.9021 - val_loss: 0.3078 - val_accuracy: 0.9010 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 149s 476ms/step - loss: 0.1379 - accuracy: 0.9434 - val_loss: 0.2107 - val_accuracy: 0.9175 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 143s 458ms/step - loss: 0.1039 - accuracy: 0.9569 - val_loss: 0.3168 - val_accuracy: 0.9285 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 145s 462ms/step - loss: 0.0950 - accuracy: 0.9644 - val_loss: 0.3555 - val_accuracy: 0.8790 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 144s 459ms/step - loss: 0.0596 - accuracy: 0.9770 - val_loss: 0.1516 - val_accuracy: 0.9525 - lr: 2.0000e-04


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 6s 101ms/step - loss: 0.4813 - accuracy: 0.8660
Test Loss: 0.4813050329685211
Test Accuracy: 0.8659999966621399


Test 5

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=45,  # Increase rotation range
    width_shift_range=0.3,  # Increase width shift range
    height_shift_range=0.3,  # Increase height shift range
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 6266s 20s/step - loss: 0.6096 - accuracy: 0.6650 - val_loss: 0.5322 - val_accuracy: 0.7275 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 158s 505ms/step - loss: 0.5352 - accuracy: 0.7245 - val_loss: 0.5130 - val_accuracy: 0.7430 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 158s 505ms/step - loss: 0.5278 - accuracy: 0.7296 - val_loss: 0.4908 - val_accuracy: 0.7580 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 157s 503ms/step - loss: 0.5260 - accuracy: 0.7337 - val_loss: 0.5003 - val_accuracy: 0.7490 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 154s 492ms/step - loss: 0.5133 - accuracy: 0.7407 - val_loss: 0.4882 - val_accuracy: 0.7600 - lr: 0.0010


In [ ]:
# Fine-tune more layers of the model
for layer in model.layers[-150:]:  # Fine-tune more layers
    layer.trainable = True

In [ ]:
# Adjust learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with fine-tuning
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
313/313 [==============================] - 183s 508ms/step - loss: 0.2314 - accuracy: 0.9078 - val_loss: 0.3032 - val_accuracy: 0.9005 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 156s 498ms/step - loss: 0.1294 - accuracy: 0.9469 - val_loss: 0.3021 - val_accuracy: 0.9225 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 159s 508ms/step - loss: 0.1098 - accuracy: 0.9541 - val_loss: 0.3475 - val_accuracy: 0.9275 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 162s 518ms/step - loss: 0.1001 - accuracy: 0.9622 - val_loss: 0.2003 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 156s 497ms/step - loss: 0.0880 - accuracy: 0.9659 - val_loss: 0.2043 - val_accuracy: 0.9400 - lr: 0.0010


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 1045s 17s/step - loss: 0.5061 - accuracy: 0.8420
Test Loss: 0.506075918674469
Test Accuracy: 0.8420000076293945


Test 6

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 8000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 1000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 1000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 8000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with dropout regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
250/250 [==============================] - 4648s 19s/step - loss: 0.6124 - accuracy: 0.6541 - val_loss: 0.5176 - val_accuracy: 0.7550 - lr: 0.0010
Epoch 2/5
250/250 [==============================] - 117s 466ms/step - loss: 0.5423 - accuracy: 0.7235 - val_loss: 0.5008 - val_accuracy: 0.7530 - lr: 0.0010
Epoch 3/5
250/250 [==============================] - 118s 474ms/step - loss: 0.5207 - accuracy: 0.7446 - val_loss: 0.5219 - val_accuracy: 0.7480 - lr: 0.0010
Epoch 4/5
250/250 [==============================] - 116s 465ms/step - loss: 0.5096 - accuracy: 0.7469 - val_loss: 0.4724 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 5/5
250/250 [==============================] - 115s 461ms/step - loss: 0.5116 - accuracy: 0.7475 - val_loss: 0.4860 - val_accuracy: 0.7650 - lr: 0.0010


In [ ]:
# Fine-tune more layers of the model
for layer in model.layers[-100:]:
    layer.trainable = True

In [ ]:
# Adjust learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with fine-tuning
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/5
250/250 [==============================] - 134s 469ms/step - loss: 0.2631 - accuracy: 0.8944 - val_loss: 0.2882 - val_accuracy: 0.8950 - lr: 0.0010
Epoch 2/5
250/250 [==============================] - 115s 461ms/step - loss: 0.1617 - accuracy: 0.9385 - val_loss: 0.4227 - val_accuracy: 0.8820 - lr: 0.0010
Epoch 3/5
250/250 [==============================] - 115s 462ms/step - loss: 0.1267 - accuracy: 0.9501 - val_loss: 0.2433 - val_accuracy: 0.9330 - lr: 0.0010
Epoch 4/5
250/250 [==============================] - 115s 462ms/step - loss: 0.1070 - accuracy: 0.9590 - val_loss: 0.3691 - val_accuracy: 0.9080 - lr: 0.0010
Epoch 5/5
250/250 [==============================] - 114s 455ms/step - loss: 0.1146 - accuracy: 0.9560 - val_loss: 0.2622 - val_accuracy: 0.9140 - lr: 0.0010


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 512s 16s/step - loss: 0.3794 - accuracy: 0.8670
Test Loss: 0.3794298768043518
Test Accuracy: 0.8669999837875366


Test 7

Original EfficientNetB0 with modified numbers of sample

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 8000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 1000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))  # EfficientNet input size
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
# Define the EfficientNet model architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
250/250 [==============================] - 494s 310ms/step - loss: 0.1709 - accuracy: 0.9295 - val_loss: 0.3025 - val_accuracy: 0.9250
Epoch 2/5
250/250 [==============================] - 38s 108ms/step - loss: 0.0758 - accuracy: 0.9694 - val_loss: 0.2689 - val_accuracy: 0.9230
Epoch 3/5
250/250 [==============================] - 38s 108ms/step - loss: 0.0462 - accuracy: 0.9827 - val_loss: 0.1882 - val_accuracy: 0.9450
Epoch 4/5
250/250 [==============================] - 38s 108ms/step - loss: 0.0448 - accuracy: 0.9818 - val_loss: 0.4939 - val_accuracy: 0.8360
Epoch 5/5
250/250 [==============================] - 38s 109ms/step - loss: 0.0369 - accuracy: 0.9861 - val_loss: 0.1849 - val_accuracy: 0.9530


In [ ]:
# Evaluate the model
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 1000)
test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 113s 4s/step - loss: 0.4388 - accuracy: 0.8410
Test Loss: 0.4387854337692261
Test Accuracy: 0.8410000205039978


V8 ResNet 101

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    all_samples, labels = shuffle(all_samples, labels)
    return all_samples, labels

In [ ]:
# Load data after sampling
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Data preprocessing
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.resnet.preprocess_input(image)
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_samples, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=len(train_samples)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_samples, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((test_samples, test_labels))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(32)

In [ ]:
# Define the ResNet101 model architecture
base_model = ResNet101(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

171446536/171446536 [==============================] - 5s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_dataset,
          epochs=5,
          validation_data=val_dataset)

Epoch 1/5
313/313 [==============================] - 865s 558ms/step - loss: 0.5073 - accuracy: 0.7643 - val_loss: 0.9388 - val_accuracy: 0.8800
Epoch 2/5
313/313 [==============================] - 76s 189ms/step - loss: 0.2172 - accuracy: 0.9143 - val_loss: 0.3767 - val_accuracy: 0.8725
Epoch 3/5
313/313 [==============================] - 76s 189ms/step - loss: 0.1500 - accuracy: 0.9379 - val_loss: 0.4613 - val_accuracy: 0.8415
Epoch 4/5
313/313 [==============================] - 75s 189ms/step - loss: 0.1287 - accuracy: 0.9486 - val_loss: 0.2167 - val_accuracy: 0.9160
Epoch 5/5
313/313 [==============================] - 76s 188ms/step - loss: 0.1088 - accuracy: 0.9542 - val_loss: 0.3556 - val_accuracy: 0.9115


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 130s 2s/step - loss: 0.5077 - accuracy: 0.8010
Test Loss: 0.5077254772186279
Test Accuracy: 0.8009999990463257
